# Imports

In [17]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

# Gradientenverfahren 1D

#### Define function and the derivative

In [18]:
# Funktion definieren
def fn1d(x: float) -> float:
    return x**5 - 3 * x**3 + x**2 + 2


# Händisch Ableitung bestimmen
def fn1d_prime(x: float) -> float:
    return 5 * x**4 - 9 * x**2 + 2 * x

#### Gradient descent

In [19]:
# Lokales Minimum finden mit Gradientenverfahren
x_start_d1 = 2  # Startpunkt
lr = 1e-2  # Schrittgröße (aka. Learning Rate)
num_rep_1d = 10
grad_diff = fn1d_prime(x_start_d1)
change_1d = True
ys = []
xs = []
while change_1d is True:
    for i in range(num_rep_1d):
        if np.absolute(grad_diff / (i + 1)) >= 1e-1:
            grad = fn1d_prime(x_start_d1)
            grad_diff += grad
            ys.append(fn1d(x_start_d1))
            xs.append(x_start_d1)
            x_start_d1 = x_start_d1 - lr * grad  # optimize
        else:
            change_1d = False
    grad_diff = grad

# Plot the Function

In [ ]:
xmin = -2
xmax = 2
x = np.arange(xmin, xmax, 0.01)
y = [fn1d(val) for val in x]

df = pd.DataFrame(dict(x=x, y=y))
fig = px.line(df, x="x", y="y", title="1-dimensional Graph")

fig.show()

# Animate the process of Gradient descent 

In [ ]:
x = np.arange(xmin, xmax, 0.01)
y = [fn1d(val) for val in x]

fig = go.Figure(
    data=[
        go.Scatter(
            x=x,
            y=y,
            mode="lines",
            line=dict(color="green", width=1),
        ),
        go.Scatter(
            x=[xs[0]],
            y=[ys[0]],
            mode="markers",
            marker=dict(color="red", size=10),
        ),
    ]
)

fig.update_layout(
    width=1200,
    height=800,
    xaxis=dict(range=(np.min(x) - 1, np.max(x) + 1), autorange=False, zeroline=False),
    yaxis=dict(range=(np.min(y) - 1, np.max(y) + 1), autorange=False, zeroline=False),
    title_text="Gradient Descent Animation",
    updatemenus=[
        dict(
            type="buttons",
            buttons=[
                dict(
                    args=[
                        None,
                        {
                            "frame": {"duration": 500, "redraw": False},
                            "fromcurrent": True,
                            "transition": {"duration": 1000},
                        },
                    ],
                    label="Play",
                    method="animate",
                )
            ],
        )
    ],
)

fig.update(
    frames=[
        go.Frame(data=[go.Scatter(x=[xs[k]], y=[ys[k]])], traces=[1])
        for k in range(len(ys))
    ]
)

fig.show()

# Gradientenverfahren 2D

In [22]:
# Funktion definieren
def fn2d(x, y) -> float:
    return np.sqrt(np.pow(x, 2) + np.pow(y, 2))


# Ableitung
def fn2d_prime(x: float, y: float) -> np.ndarray:  # gibt 2D Array zurück
    partial_x = 2 * x  # leite f nach x ab, lass y konstant
    partial_y = 2 * y  # leite f nach y ab, lass x konstant
    return np.array([[partial_x], [partial_y]], ndmin=2)


# Lokales Minimum finden mit Gradientenverfahren
x_start = 2.0  # Startpunkt
y_start = 2.0
lr = 1e-1  # Schrittgröße
num_rep_2d = 10
grad_xy_diff = fn2d_prime(x_start, y_start)
change_2d = True
xs = [x_start]
ys = [y_start]
zs = [fn2d(x_start, y_start)]


while change_2d == True:
    for i in range(num_rep_2d):
        if np.absolute(np.mean(grad_xy_diff)) / (i + 1) >= 1e-3 and i == 0:
            grad_xy = fn2d_prime(x_start, y_start)
            x_start = x_start - lr * grad_xy[0, 0]
            y_start = y_start - lr * grad_xy[1, 0]
            zs.append(fn2d(x_start, y_start))
            xs.append(x_start)
            ys.append(y_start)
        elif np.absolute(np.mean(grad_xy_diff)) / (i + 1) >= 1e-3:
            grad_xy = fn2d_prime(x_start, y_start)
            grad_xy_diff[0, 0] += grad_xy[0, 0]
            grad_xy_diff[1, 0] += grad_xy[1, 0]
            x_start = x_start - lr * grad_xy[0, 0]
            y_start = y_start - lr * grad_xy[1, 0]
            zs.append(fn2d(x_start, y_start))
            xs.append(x_start)
            ys.append(y_start)
        else:
            change_2d = False
            zs.append(fn2d(x_start, y_start))
            xs.append(x_start)
            ys.append(y_start)
    grad_xy_diff[0, 0], grad_xy_diff[1, 0] = grad_xy[0, 0], grad_xy[1, 0]
    # print(x_start, grad_xy[0, 0], y_start, grad_xy[1, 0])

# Plot the Surface

In [ ]:
import plotly.io as pio

pio.renderers.default = "notebook"

xy_min = -2
xy_max = 2
r = np.arange(xy_min, xy_max, 0.01)
x, y = np.meshgrid(r, r)

vals = fn2d(x, y)

fig2 = go.Figure(data=[go.Surface(z=vals, x=x, y=y)])

fig2.update_layout(title="Surface Plot", autosize=True)


fig2.show()

# Plot the process of Gradient Descent

In [ ]:
pio.renderers.default = "notebook"

r = np.arange(xy_min, xy_max, 0.01)
x, y = np.meshgrid(r, r)

vals = fn2d(x, y)

fig = go.Figure(
    data=[
        go.Surface(z=vals, x=x, y=y),
        go.Scatter3d(
            x=[xs[0]],
            y=[ys[0]],
            z=[zs[0]],
            mode="markers",
            marker=dict(color="cyan", size=5),
        ),
    ]
)

fig.update_layout(
    scene=dict(
        xaxis=dict(
            range=(np.min(x) - 2, np.max(x) + 2), autorange=False, zeroline=False
        ),
        yaxis=dict(
            range=(np.min(y) - 2, np.max(y) + 2), autorange=False, zeroline=False
        ),
        zaxis=dict(range=(np.min(zs), np.max(zs) + 2), autorange=False, zeroline=False),
    ),
    title_text="Gradient Descent Animation",
    updatemenus=[
        dict(
            type="buttons",
            buttons=[
                dict(
                    args=[
                        None,
                        {
                            "frame": {"duration": 500, "redraw": True},
                            "fromcurrent": True,
                            "transition": {"duration": 1000},
                        },
                    ],
                    label="Play",
                    method="animate",
                )
            ],
        )
    ],
)

fig.update(
    frames=[
        go.Frame(data=[go.Scatter3d(x=[xs[k]], y=[ys[k]], z=[zs[k]])], traces=[1])
        for k in range(len(zs))
    ]
)

fig.show()